In [1]:
import pandas as pd
from nltk.tokenize import sent_tokenize, PunktSentenceTokenizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
import math
from rouge import Rouge
from tqdm.notebook import tqdm
import seaborn as sns
import numpy as np
import pickle
sns.set()

In [2]:
def show(x):
    print(x.shape)
    return x.head()

In [3]:
references = pd.read_csv("../data/references.csv").dropna()
show(references)
references.head(10)

(5241, 3)


,global_reference_id,title,abstract
0,0,Distributed Representations of Words and Phras...,The recently introduced continuous Skip-gram m...
1,1,BERT: Pre-training of Deep Bidirectional Trans...,We introduce a new language representation mod...
2,2,Attention Is All You Need,The dominant sequence transduction models are ...
3,3,GloVe : Global Vectors for Word Representation,Recent methods for learning vector space repre...
4,4,Adam: A Method for Stochastic Optimization,"We introduce Adam, an algorithm for first-orde..."
6,6,Deep Contextualized Word Representations,We introduce a new type of deep contextualized...
7,7,Neural Machine Translation By Jointly Learning...,Neural machine translation is a recently propo...
8,8,Efficient Estimation of Word Representations i...,We propose two novel model architectures for c...
9,9,Deep Residual Learning for Image Recognition,Deeper neural networks are more difficult to t...
11,11,Distributed Representations of Sentences and D...,Many machine learning algorithms require the i...


In [4]:
papers = pd.read_csv("../data/papers.csv")
show(papers)

(1091, 4)


,paper_id,title,abstract,text
0,0,IDST at TREC 2019 Deep Learning Track: Deep Ca...,This paper describes our participation in the ...,"KEYWORDS cascade ranking, pre-trained language..."
1,1,BatchBALD: Efficient and Diverse Batch Acquisi...,"We develop BatchBALD, a tractable approximatio...",A key problem in deep learning is data efficie...
2,2,A Sentence Compression Based Framework to Quer...,We consider the problem of using sentence comp...,Proceedings of the 51st Annual Meeting of the ...
3,3,DR-BiLSTM: Dependent Reading Bidirectional LST...,We present a novel deep learning architecture ...,Natural Language Inference (NLI; a.k.a. Recogn...
4,4,On Minimizing Cost in Legal Document ReviewWor...,Technology-assisted review (TAR) refers to hum...,CCS CONCEPTS • Information systems → Informati...


In [5]:
citations = pd.read_csv("../data/citations.csv")
show(citations)

(56746, 6)


,paper_id,internal_reference_id,global_reference_id,context,start_offset,end_offset
0,0,1,1,Our approach is mainly based on the BERT langu...,56,59
1,0,8,475,Different from many other ranking methods whic...,216,219
2,0,5,155,"Moreover, for full ranking subtask, we use a s...",78,81
3,0,6,25,The proposed model is based on the pointer-gen...,59,62
4,0,6,25,"Following [7], we also use a soft switch to ch...",10,13


# retrieving sentences

In [6]:
query_papers = citations.paper_id.value_counts().reset_index()\
.rename(columns = {"index":"paper_id", "paper_id":"references"})\
.merge(papers.drop(columns = ["title", "abstract"]))
show(query_papers)

(1091, 3)


,paper_id,references,text
0,482,718,This survey provides a synthesis of existing w...
1,647,718,This survey provides a synthesis of existing w...
2,595,599,commercial world. To provide an overview of th...
3,650,563,1 Interactive Clustering: A Comprehensive Revi...
4,372,491,"In this survey, we provide a synthesis of exis..."


In [7]:
papers.query("paper_id == 191")

,paper_id,title,abstract,text
191,191,The Benefits of Word Embeddings Features for A...,This study investigates the use of unsupervise...,Active learning (AL) has recently received con...


In [8]:
query_references = citations.global_reference_id.value_counts()\
.reset_index()\
.rename(columns = {"index":"global_reference_id", "global_reference_id":"citations"})\
.merge(references.dropna().drop(columns = "title"), right_index = True, left_on = "global_reference_id")
show(query_references)

(4193, 5)


,global_reference_id,global_reference_id_x,citations,global_reference_id_y,abstract
0,1,1,497,1,We introduce a new language representation mod...
1,2,2,257,2,The dominant sequence transduction models are ...
2,0,0,247,0,The recently introduced continuous Skip-gram m...
3,3,3,215,3,Recent methods for learning vector space repre...
4,8,8,184,8,We propose two novel model architectures for c...


In [9]:
query_citations = citations.merge(query_references)[["paper_id", "global_reference_id"]].drop_duplicates()
show(query_citations)

(11310, 2)


,paper_id,global_reference_id
0,0,1
2,7,1
6,8,1
12,12,1
18,17,1


In [10]:
tokenizer = PunktSentenceTokenizer()
# tokenizer = PunktSentenceTokenizer(train_text = papers.text.sum())
tokenizer

In [11]:
# model = CountVectorizer()
# model = TfidfVectorizer()
# model.fit(query_references.abstract)

In [12]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [15]:
query_embeddings = model.encode(query_references.abstract.tolist(), show_progress_bar = True, device = "cuda")
print(query_embeddings.shape)

Batches:   0%|          | 0/132 [00:00<?, ?it/s]

(4193, 384)


In [16]:
sentence_embeddings = []
for p in tqdm(query_papers.text):
    s = tokenizer.tokenize(p)
    sentence_embeddings.append((np.array(s), model.encode(s, show_progress_bar = True, device = "cuda")))
print(len(sentence_embeddings))
print(len(sentence_embeddings[0][0]), sentence_embeddings[0][1].shape)

  0%|          | 0/1091 [00:00<?, ?it/s]

Batches:   0%|          | 0/187 [00:00<?, ?it/s]

Batches:   0%|          | 0/187 [00:00<?, ?it/s]

Batches:   0%|          | 0/17 [00:00<?, ?it/s]

Batches:   0%|          | 0/42 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/14 [00:00<?, ?it/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

1091
5963 (5963, 384)


In [17]:
query_ids = query_references\
.global_reference_id\
.reset_index(drop = True)\
.reset_index()\
.set_index("global_reference_id")["index"]
show(query_ids)

(4193,)


global_reference_id
1    0
2    1
0    2
3    3
8    4
Name: index, dtype: int64

In [18]:
paper_ids = query_papers\
.paper_id\
.reset_index(drop = True)\
.reset_index()\
.set_index("paper_id")["index"]
show(paper_ids)

(1091,)


paper_id
482    0
647    1
595    2
650    3
372    4
Name: index, dtype: int64

In [ ]:
data = query_citations.to_dict("records")#[:5]
sentence_scores = []
for d in tqdm(data):
    paper = d["paper_id"]
    reference = d["global_reference_id"]
    qe = query_embeddings[query_ids[reference]]
    sentences, embeddings = sentence_embeddings[paper_ids[paper]]
#     print(qe.shape)
#     print(embeddings.shape)
#     sim = cosine_similarity(qe.toarray(), embeddings.toarray()).flatten()
#     sim = (qe.toarray() @ embeddings.toarray().T).flatten()
    sim = (qe @ embeddings.T).flatten()
    df = pd.DataFrame({"text":sentences, "relevance_to_query":sim})\
    .assign(paper_id = paper, global_reference_id = reference)
    sentence_scores.append(df)
sentence_scores = pd.concat(sentence_scores, axis = 0)
show(sentence_scores)

  0%|          | 0/11310 [00:00<?, ?it/s]

In [ ]:
sentence_scores.groupby(["paper_id", "global_reference_id"]).relevance_to_query.mean().plot.hist()
plt.xlabel("Average relevance to query")
plt.show()

In [ ]:
queries = sentence_scores.query("relevance_to_query > 0.1")\
.groupby(["paper_id", "global_reference_id"])\
.text.sum()\
.to_frame()\
.reset_index()\
.merge(references.drop(columns = ["title"]))\
.rename(columns = {"text":"hypothesis", "abstract":"reference"})\
.dropna()\
.reset_index(drop = True)

show(queries)

# rouge

In [ ]:
rouge = Rouge()
scores = rouge.get_scores(hyps = queries.hypothesis, refs = queries.reference)
print(len(scores))
scores[:5]

In [ ]:
rouge_df = pd.DataFrame.from_records(scores)
show(rouge_df)

In [ ]:
rouge_1 = pd.DataFrame.from_records(rouge_df["rouge-1"])\
.rename(columns = {"f":"rouge_1_f", "r":"rouge_1_r", "p":"rouge_1_p"})
show(rouge_1)

In [ ]:
rouge_2 = pd.DataFrame.from_records(rouge_df["rouge-2"])\
.rename(columns = {"f":"rouge_2_f", "r":"rouge_2_r", "p":"rouge_2_p"})
show(rouge_2)

In [ ]:
rouge_l = pd.DataFrame.from_records(rouge_df["rouge-l"])\
.rename(columns = {"f":"rouge_l_f", "r":"rouge_l_r", "p":"rouge_l_p"})
show(rouge_l)

In [ ]:
full_rouge = pd.concat((queries, rouge_1, rouge_2, rouge_l), axis = 1)
show(full_rouge)

In [ ]:
f, ax = plt.subplots(3, 1, sharex = True, constrained_layout=True, figsize = (12, 6))
ax[0].hist(full_rouge.rouge_1_f, bins = 50)
ax[0].set_title("Rouge-1")
ax[1].hist(full_rouge.rouge_2_f, bins = 50, color = "green")
ax[1].set_title("Rouge-2")
ax[2].hist(full_rouge.rouge_l_f, bins = 50, color = "red")
ax[2].set_title("Rouge-L")

plt.show()

In [ ]:
data = full_rouge.sort_values("rouge_1_f").reset_index()
f, ax = plt.subplots(3, 1, sharex = True, constrained_layout=True, figsize = (12, 6))
ax[0].plot(data.rouge_1_f)
ax[0].set_title("Rouge-1")
ax[1].plot(data.rouge_2_f, color = "green")
ax[1].set_title("Rouge-2")
ax[2].plot(data.rouge_l_f, color = "red")
ax[2].set_title("Rouge-L")

plt.show()

In [ ]:
sns.pairplot(full_rouge, vars = ["rouge_1_f","rouge_2_f", "rouge_l_f"])
plt.show()

In [ ]:
data = full_rouge.sort_values("rouge_1_f").reset_index()
f, ax = plt.subplots(3, 1, sharex = True, constrained_layout=True, figsize = (12, 6))
ax[0].plot(data.rouge_1_f)
ax[0].set_title("Rouge-1 F")
ax[1].plot(data.rouge_1_p)
ax[1].set_title("Rouge-1 P")
ax[2].plot(data.rouge_1_r)
ax[2].set_title("Rouge-1 R")

plt.show()

In [ ]:
sns.pairplot(full_rouge, vars = ["rouge_1_f","rouge_1_p", "rouge_1_r"])
plt.show()

# inspection

## worse

In [ ]:
full_rouge.sort_values("rouge_1_f").head()

In [ ]:
full_rouge.sort_values("rouge_1_f").head().hypothesis.tolist()

In [ ]:
full_rouge.sort_values("rouge_1_f").head().reference.tolist()

## best

In [ ]:
full_rouge.sort_values("rouge_1_f", ascending = False).head()

In [ ]:
full_rouge.sort_values("rouge_1_f", ascending = False).head().hypothesis.tolist()

In [ ]:
full_rouge.sort_values("rouge_1_f", ascending = False).head().reference.tolist()

# final performance

In [ ]:
full_rouge[["rouge_1_f", "rouge_2_f", "rouge_l_f"]].describe(percentiles = np.linspace(0, 1, 11))